In [1]:
import base64
from IPython.display import Image, display

def mm_ink(graphbytes):
  """Given a bytes object holding a Mermaid-format graph, return a URL that will generate the image."""
  base64_bytes = base64.b64encode(graphbytes)
  base64_string = base64_bytes.decode("ascii")
  return "https://mermaid.ink/img/" + base64_string

def mm_display(graphbytes):
  """Given a bytes object holding a Mermaid-format graph, display it."""
  display(Image(url=mm_ink(graphbytes)))

def mm(graph):
  """Given a string containing a Mermaid-format graph, display it."""
  graphbytes = graph.encode("ascii")
  mm_display(graphbytes)

def mm_link(graph):
  """Given a string containing a Mermaid-format graph, return URL for display."""
  graphbytes = graph.encode("ascii")
  return mm_ink(graphbytes)
  
def mm_path(path):
  """Given a path to a file containing a Mermaid-format graph, display it"""
  with open(path, 'rb') as f:
    graphbytes = f.read()
  mm_display(graphbytes)

In [ ]:
def upload_file_to_volume(local_file_path: str, volume_folder_path: str, file_name: str) -> None:
    """Upload local file to the specified volume. ."""
    
    from databricks.sdk import WorkspaceClient
    import io

    # Initialize the WorkspaceClient
    w = WorkspaceClient()

    # Create an empty folder in the volume
    w.files.create_directory(volume_folder_path)

    volume_file_path = f"{volume_folder_path}/{file_name}"  # /Volumes/main/default/my-volume/my-folder/data.csv
  
    # Upload the file to the volume
    with open(local_file_path, 'rb') as file:
        file_bytes = file.read()
        binary_data = io.BytesIO(file_bytes)
        w.files.upload(volume_file_path, binary_data, overwrite=True)


In [ ]:
def generate_table_ddl (catalog: str, schema: str, table:str) -> str:

    # Fetch table-level and column-level metadata
    table_metadata = spark.sql(f"""
        SELECT t.table_name, t.table_type, t.comment AS table_comment
        FROM {catalog}.information_schema.tables t
        WHERE t.table_schema = '{schema}' AND t.table_name = '{table}'
    """).collect()[0]

    columns_metadata = spark.sql(f"""
        SELECT column_name, data_type, is_nullable, comment
        FROM {catalog}.information_schema.columns
        WHERE table_schema = '{schema}' AND table_name = '{table}'
        ORDER BY ordinal_position
    """).collect()

    # Build CREATE TABLE statement
    ddl = f"CREATE TABLE {catalog}.{schema}.{table} (\n"
    column_lines = []

    for col in columns_metadata:
        comment = f" COMMENT '{col['comment']}'" if col['comment'] else ""
        nullable = "" if col['is_nullable'] == "YES" else " NOT NULL"
        column_lines.append(f"  {col['column_name']} {col['data_type']}{nullable}{comment}")

    ddl += ",\n".join(column_lines) + "\n)"

    # Add table comment if available
    if table_metadata['table_comment']:
        ddl += f"\nCOMMENT '{table_metadata['table_comment']}'"

    # Add table type (optional)
    ddl += f"\n-- Table Type: {table_metadata['table_type']}"

    # Output the generated DDL
    return ddl

In [19]:

import logging

# Create a module‐level logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# (Optional) add a handler if you need custom formatting or
# want to write to a file rather than stdout/stderr:
handler = logging.StreamHandler()
formatter = logging.Formatter(
    "%(asctime)s %(levelname)s %(name)s %(message)s"
)
handler.setFormatter(formatter)
logger.addHandler(handler)
